In [4]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession

In [6]:
schema = StructType([
StructField("DATE", DateType()),
StructField("COUNTRY", StringType()),
StructField("CITY", StringType()),
StructField("VALUE", DoubleType())])

spark = SparkSession.builder.appName('TSF-pollution').getOrCreate()

df = spark.read.csv("/Users/sanjju/projects/datasets/air_quality_index.csv", header = 'true', schema=schema)



In [7]:

# @udf(VectorUDT())
def toSparseVector(index, values):
    day_list_index, qty_list_values = zip(*sorted(zip(index, values)))
    #367 for bisextile year (1 to 366 +1)
    return Vectors.sparse(367, day_list_index, qty_list_values)


In [8]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import HiveContext, Window, DataFrameWriter, Row
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.classification import LinearSVC
import sys
from operator import add
from functools import reduce, wraps
import logging, time
from functools import wraps

In [9]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import udf
from pyspark.sql import functions as F

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import *
from pyspark.ml.linalg import Vectors, VectorUDT
import numpy as np

from pyspark.ml import Pipeline
from pyspark.ml.feature import PCA
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, Imputer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import *


# from features_utils import * 

In [10]:
schema = StructType([
StructField("DATE", DateType()),
StructField("COUNTRY", StringType()),
StructField("CITY", StringType()),
StructField("VALUE", DoubleType())])

In [11]:
spark = SparkSession.builder.appName('TSF').getOrCreate()

df = spark.read.csv("dataset/air_quality_index.csv", header = 'true', schema=schema)


In [12]:
df = (df
    .withColumn('yearday', F.dayofyear(F.col("DATE")))
    .withColumn('year', F.year(F.col('DATE')))
    )

self_join = (df
    .groupby("COUNTRY", "year")
    .agg(
        F.collect_list("COUNTRY").alias("qties"),
         F.collect_list("yearday").alias("days")
         )
    .withColumn("qties_vectorized", toSparseVector(F.col("days"), F.col("qties")))
    .withColumn("year_join", F.col("year") + 1)
     .select(
      F.col("SID_STORE").alias("p_id_store"), F.col("year").alias("year2"), F.col("qties_vectorized").alias("qties_vectorized"),
       F.col("days").alias("dayss"),  F.col("qties").alias("qties"),  F.col("year_join").alias("year_join")
        )
     )


df= (df
    .join(self_join
        , ([self_join.p_id_store == df.SID_STORE, self_join.year_join ==  df.year]),
        how = "left"
        )
        .withColumn("qty_reference", getReference(F.col("yearday"), F.col("qties_vectorized")))
        .withColumn('qty_reference', F.col("qty_reference").cast(DoubleType()))
        .withColumn('qty_reference', F.when(F.col("qty_reference").isNull(), F.lit(0)).otherwise(F.col("qty_reference")))
    )


TypeError: Column is not iterable

In [ ]:

dex = DayExtractor(inputCol='DATE')
mex = MonthExtractor(inputCol='DATE')
yex = YearExtractor(inputCol='DATE')
wdex = WeekDayExtractor(inputCol='DATE')
wex = WeekendExtractor()
vex = VivaldiExtractor(inputCol = "month")
mqex = MonthQuarterExtractor(inputCol = "day")
mbex = MonthBeginExtractor()
meex = MonthEndExtractor()
yqex = YearQuarterExtractor()
ydex = YearDayExtractor(inputCol='DATE')

numeric_col= ["VALUE"]
